In [10]:
from diffwave.inference import predict as diffwave_predict
import torch
import numpy as np
import IPython.display as ipd


In [11]:
model_dir = 'diffwave-ljspeech-22kHz-1000578.pt'
spectrogram_data = torch.from_numpy(np.load('mel_spectrogram_data.npy')).float().unsqueeze(0).to('cuda')
spectrogram = spectrogram_data
audio, sample_rate = diffwave_predict(spectrogram, model_dir, fast_sampling=True)

audio_for_playback = audio.squeeze().cpu()
ipd.display(ipd.Audio(audio_for_playback, rate=sample_rate))